In [5]:
import datasets
ds = datasets.load_dataset('deepmind/code_contests', split='train')

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
def map_code_contests(example):
    question = example['description']
    test_inputs = [x.strip().split('\n') for x in example['private_tests']['input']]
    test_outputs = [x.strip() for x in example['private_tests']['output']]
    max_time = float(example['time_limit']['seconds'])
    
    return {
        'question': question,
        'test_inputs': test_inputs,
        'test_outputs': test_outputs,
        'max_time': max_time
    }
ds = ds.select(range(100))
ds = ds.filter(lambda ex: ex['difficulty']  == 7 and '<image>' not in ex['description']) # filter easy samples 
ds = ds.map(map_code_contests, remove_columns=ds.column_names)
        

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{'input': ['51\n882889888888689888850888388887688788888888888858888\n', '55\n7271714707719515303911625619272900050990324951111943573\n', '72\n888488888888823288848804883838888888887888888888228888218488897809784868\n', '65\n44542121362830719677175203560438858260878894083124543850593761845\n', '54\n438283821340622774637957966575424773837418828888614203\n', '100\n1976473621569903172721407763737179639055561746310369779167351419713916160700096173622427077757986026\n', '100\n2833898888858387469888804083887280788584887487186899808436848018181838884988432785338497585788803883\n', '42\n885887846290886288816884858898812858495482\n', '75\n878909759892888846183608689257806813376950958863798487856148633095072259838\n', '11\n55814018693\n', '31\n0868889888343881888987888838808\n', '21\n888888888888000000000\n', '62\n18888883884288488882387888486858887882838885288886472818688888\n', '77\n11111111111111111111111111111111111111111111111111111111111111111111111111111\n', '30\n88888888888888888888888888

In [7]:
list(ds[0].keys())

['question', 'test_inputs', 'test_outputs', 'max_time']

In [ ]:
import io
import sys

def test_solution(test_case):
    test_input = test_case['test_inputs']
    test_output = test_case['test_outputs']
    # Define the solution code as a string
    solution = test_case['solution']
    res = []
    for i in range(len(test_input)):
        # Redirect stdout to capture print statements in the solution code
        output_capture = io.StringIO()
        original_stdout = sys.stdout
        sys.stdout = output_capture
        inp = iter(test_input[i])
        # Execute the solution code with a custom input function
        try:
            # Set up a local environment with a mock input function
            local_vars = {"input": lambda: next(inp)}

            # Execute the solution in this environment
            exec(solution, {}, local_vars)
        finally:
            # Restore the original stdout
            sys.stdout = original_stdout

        # Retrieve the captured output
        captured_output = output_capture.getvalue().strip()

        # Compare the captured output to the expected output
        if captured_output == test_output[i]:
            res.append(1)
        else:
            #print(test_case['solution'], end='------------------\n')

            res.append(0)
    return sum(res) / len(res)

for i,c in enumerate(modified_dataset):
    print(f"case number {i} has {test_solution(c)*100} % correct tests")

In [ ]:
print('\n'.join([f"{k}: {v}" for k,v in modified_dataset[0].items()]))